In [1]:
# Library imports
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time

# Aliases
keras = tf.keras

In [2]:
# Local imports
from utils import gpu_grow_memory, TimeHistory, EpochLoss
from utils import load_vartbl, save_vartbl, plot_style

from polar_coord import make_data_sin, make_dataset_sin
from polar_coord import make_model_sin_math

In [3]:
# Grow GPU memory (must be first operation in TF)
gpu_grow_memory()

In [4]:
# Plot style 
plot_style()

In [5]:
# Make datasets for sin
n = 3600
ds_p2c, ds_c2p, d2_p2p, ds_c2c = make_dataset_sin(n)

In [6]:
# Unpack one example batch
theta, y = list(ds_p2c.take(1))[0]
theta, y = theta.numpy(), y.numpy()

# Review data contents
print(f'theta: min={np.min(theta):+8.5}, max={np.max(theta):+8.5}')
print(f'    y: min={np.min(y):+8.5}, max={np.max(y):+8.5}')

theta: min= -1.5708, max= +1.5708
    y: min=    -1.0, max=    +1.0


In [7]:
# Mathematical models
math_p2c, math_c2p, math_p2p, math_c2c = make_model_sin_math()

In [8]:
# Set loss function and optimizer
loss = keras.losses.MeanSquaredError()
optimizer = keras.optimizers.Adam()

In [9]:
# Compile the math models
math_p2c.compile(loss=loss, optimizer=optimizer)
math_c2p.compile(loss=loss, optimizer=optimizer)
math_p2p.compile(loss=loss, optimizer=optimizer)
math_c2c.compile(loss=loss, optimizer=optimizer)

In [10]:
# Evaluate the p2c model
math_p2c.evaluate(ds_p2c)

1/1 [==============================] - 0s 55ms/step - loss: 4.6015e-16


4.601469428267544e-16

In [11]:
# Evaluate the c2p model
math_c2p.evaluate(ds_c2p)

1/1 [==============================] - 0s 45ms/step - loss: 6.4945e-13


6.494470217686954e-13

In [12]:
# Evaluate the p2p model
math_p2p.evaluate(theta, theta)

3601/3601 [==============================] - 0s 41us/sample - loss: 6.4904e-13


6.490446467481708e-13

In [13]:
# Evaluate the c2c model
math_c2c.evaluate(y, y)

3601/3601 [==============================] - 0s 44us/sample - loss: 7.0153e-16


7.01525834862032e-16

In [14]:
def make_model_sin():
    """Neural net model of y = sin(theta)"""
    # Input layer
    theta = keras.Input(shape=(1,), name='theta')
    
    # Dense feature layers
    phi_1 = keras.layers.Dense(units=16, activation='tanh')(theta)
    phi_2 = keras.layers.Dense(units=16, activation='tanh')(phi_1)
    
    # Output layer
    y = keras.layers.Dense(units=1)(phi_2)
    
    # Wrap into a model
    model = keras.Model(inputs=theta, outputs=y)
    return model

In [15]:
# Build the NN model of y=sin(theta)
model_sin = make_model_sin()

In [16]:
# Compile
model_sin.compile(loss=loss, optimizer=optimizer)

In [37]:
class EpochLoss(tf.keras.callbacks.Callback):
    """Log the loss every N epochs"""
    def __init__(self, interval=10):
        super(EpochLoss, self).__init__()
        self.interval = interval

    def log_to_screen(self, epoch, logs):
        loss = logs['loss']
        print(f'Epoch {epoch:04}; loss {loss:5.2e}')            
        
    def on_epoch_end(self, epoch, logs=None):
        if epoch % self.interval == 0:
            self.log_to_screen(epoch, logs)
            
    def on_train_end(self, logs=None):
        loss = logs['loss']
        print(f'Train end; loss {loss:5.2e}')

In [38]:
# Callbacks for NN models
cb_log = EpochLoss(interval=50)
cb_time = TimeHistory()
callbacks = [cb_log, cb_time]

In [39]:
hist = model_sin.fit(ds_p2c, epochs=200, callbacks=callbacks, verbose=0)

W0621 17:17:01.135236 22112 training_utils.py:1436] Expected a shuffled dataset but input dataset `x` is not shuffled. Please invoke `shuffle()` on input dataset.


Epoch 0000; loss 3.98e-05
Epoch 0050; loss 3.86e-05
Epoch 0100; loss 3.74e-05
Epoch 0150; loss 3.61e-05


TypeError: 'NoneType' object is not subscriptable

In [ ]:
hist.history['loss']

In [ ]:
hist.history.keys()